# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.4.4/sql-programming-guide.html

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.4.4/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.4.4/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

### If you are running this notebook in Google Colab

Copy the following to a code cell and run it. It will install and set up Spark for you.

```python
!pip install pyspark==3.1.1
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.ui.port", "4050").getOrCreate() ## For ngrok to tunnel to
```

In [1]:
!pip install pyspark==3.1.1
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.ui.port", "4050").getOrCreate() ## For ngrok to tunnel to


     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 18.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=f16b216b315c08a08798c8f8cb580b1153199b724385db5ca1edfd0dc751e0dc
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


#### Passing other options to spark session:
    
    

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.driver.host', '435c62bc596a'),
 ('spark.executor.id', 'driver'),
 ('spark.sql.warehouse.dir', 'file:/content/spark-warehouse'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.ui.port', '4050'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1616177659680'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '35025'),
 ('spark.app.startTime', '1616177658180'),
 ('spark.ui.showConsoleProgress', 'true')]

We can check option values in the resulting session like this:

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [8]:
import random

random.seed(7)
ids = range(15)
species = random.choices(['capibara', 'naked mole rat', 'axolotl', 'flying fox'], k=15)

data = list(zip(ids, species))
data

[(0, 'naked mole rat'),
 (1, 'capibara'),
 (2, 'axolotl'),
 (3, 'capibara'),
 (4, 'axolotl'),
 (5, 'naked mole rat'),
 (6, 'capibara'),
 (7, 'axolotl'),
 (8, 'capibara'),
 (9, 'naked mole rat'),
 (10, 'capibara'),
 (11, 'capibara'),
 (12, 'naked mole rat'),
 (13, 'flying fox'),
 (14, 'capibara')]

In [10]:
df = spark.createDataFrame(data)
df

DataFrame[_1: bigint, _2: string]

In [11]:
df.show()

+---+--------------+
| _1|            _2|
+---+--------------+
|  0|naked mole rat|
|  1|      capibara|
|  2|       axolotl|
|  3|      capibara|
|  4|       axolotl|
|  5|naked mole rat|
|  6|      capibara|
|  7|       axolotl|
|  8|      capibara|
|  9|naked mole rat|
| 10|      capibara|
| 11|      capibara|
| 12|naked mole rat|
| 13|    flying fox|
| 14|      capibara|
+---+--------------+



In [12]:
df.take(5)

[Row(_1=0, _2='naked mole rat'),
 Row(_1=1, _2='capibara'),
 Row(_1=2, _2='axolotl'),
 Row(_1=3, _2='capibara'),
 Row(_1=4, _2='axolotl')]

In [13]:
first_row = df.first()
first_row

Row(_1=0, _2='naked mole rat')

In [17]:
first_row['_2']

'naked mole rat'

### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

This is what we did on the previous notebook:

```python
import re

def remove_punctuation(line):
    return re.sub('[^a-z0-9 ]', '', line.lower())

shakespeare = spark.sparkContext.textFile('shakespeare.txt')

freqs = shakespeare.map(remove_punctuation)\
                   .flatMap(str.split)\
                   .map(lambda word: (word, 1))\
                   .reduceByKey(lambda x, y: x + y)

```

We can put that on a DataFrame in order to further analyze it:

### Inferring and specifying schemas

In [19]:
df = spark.createDataFrame(data, schema=['id', 'species'])
df

DataFrame[id: bigint, species: string]

In [24]:
from pyspark.sql import types

types.StringType()
types.IntegerType()
types.ShortType()

ShortType

In [25]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- species: string (nullable = true)



#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [28]:
df = spark.createDataFrame(data, schema='id SHORT, species STRING')
df.printSchema()
df.show()

root
 |-- id: short (nullable = true)
 |-- species: string (nullable = true)

+---+--------------+
| id|       species|
+---+--------------+
|  0|naked mole rat|
|  1|      capibara|
|  2|       axolotl|
|  3|      capibara|
|  4|       axolotl|
|  5|naked mole rat|
|  6|      capibara|
|  7|       axolotl|
|  8|      capibara|
|  9|naked mole rat|
| 10|      capibara|
| 11|      capibara|
| 12|naked mole rat|
| 13|    flying fox|
| 14|      capibara|
+---+--------------+



In [29]:
df.schema

StructType(List(StructField(id,ShortType,true),StructField(species,StringType,true)))

#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [33]:
coupons = spark.read.csv('coupon150720.csv.gz')
coupons.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [34]:
spark.read.json
spark.read.parquet

<bound method DataFrameReader.parquet of <pyspark.sql.readwriter.DataFrameReader object at 0x7f81033ae8d0>>

### Basic operations with DataFrames

In [37]:
df.select('id').show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+



### Filtering and selecting

Syntax inspired in SQL.

In [41]:
from pyspark.sql import Column

df['id'] > 5

Column<'(id > 5)'>

If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [43]:
df.filter(df['id'] > 5).show()

+---+--------------+
| id|       species|
+---+--------------+
|  6|      capibara|
|  7|       axolotl|
|  8|      capibara|
|  9|naked mole rat|
| 10|      capibara|
| 11|      capibara|
| 12|naked mole rat|
| 13|    flying fox|
| 14|      capibara|
+---+--------------+



That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

In [44]:
df.filter('id' > 5)

TypeError: ignored

`where` is exactly synonimous with `filter`

In [46]:
df.where(df['id'] > 5).show()

+---+--------------+
| id|       species|
+---+--------------+
|  6|      capibara|
|  7|       axolotl|
|  8|      capibara|
|  9|naked mole rat|
| 10|      capibara|
| 11|      capibara|
| 12|naked mole rat|
| 13|    flying fox|
| 14|      capibara|
+---+--------------+



In [49]:
df.where((df['id']> 5) & (df['id']<10)).show()

+---+--------------+
| id|       species|
+---+--------------+
|  6|      capibara|
|  7|       axolotl|
|  8|      capibara|
|  9|naked mole rat|
+---+--------------+



In [50]:
df.filter('id > 5')

DataFrame[id: smallint, species: string]

In [52]:
df[df['id'] > 5]

DataFrame[id: smallint, species: string]

A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

In [54]:
df['id'].show()

TypeError: ignored

In [56]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-03-19 19:24:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.192.67.182, 52.206.108.108, 3.216.44.25, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.192.67.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  65.6MB/s    in 0.2s    

2021-03-19 19:24:20 (65.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
https://4ea560865ad0.ngrok.io


In [57]:
df.rdd

MapPartitionsRDD[90] at javaToPython at NativeMethodAccessorImpl.java:0

#### Exercise

Extract all zoo animal ids which correspond to capibaras

In [60]:
df.select('id')\
  .where(df['species']=='capibara')\
  .show()

+---+
| id|
+---+
|  1|
|  3|
|  6|
|  8|
| 10|
| 11|
| 14|
+---+



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we can not assign to them. We need to create new DataFrames with the appropriate columns.

In [64]:
from pyspark.sql import functions

df.withColumn('whateva', functions.sqrt('id')).show()

+---+--------------+------------------+
| id|       species|           whateva|
+---+--------------+------------------+
|  0|naked mole rat|               0.0|
|  1|      capibara|               1.0|
|  2|       axolotl|1.4142135623730951|
|  3|      capibara|1.7320508075688772|
|  4|       axolotl|               2.0|
|  5|naked mole rat|  2.23606797749979|
|  6|      capibara| 2.449489742783178|
|  7|       axolotl|2.6457513110645907|
|  8|      capibara|2.8284271247461903|
|  9|naked mole rat|               3.0|
| 10|      capibara|3.1622776601683795|
| 11|      capibara|   3.3166247903554|
| 12|naked mole rat|3.4641016151377544|
| 13|    flying fox| 3.605551275463989|
| 14|      capibara|3.7416573867739413|
+---+--------------+------------------+



In [67]:
import math

math.log1p(df['id'])

TypeError: ignored

### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [70]:
my_new_udf = functions.udf(math.log1p)
my_new_udf(df['id'])

Column<'log1p(id)'>

In [72]:
df.withColumn('yeah_baby',
              my_new_udf(df['id'])).show()

+---+--------------+------------------+
| id|       species|         yeah_baby|
+---+--------------+------------------+
|  0|naked mole rat|               0.0|
|  1|      capibara|0.6931471805599453|
|  2|       axolotl|1.0986122886681096|
|  3|      capibara|1.3862943611198906|
|  4|       axolotl|1.6094379124341003|
|  5|naked mole rat| 1.791759469228055|
|  6|      capibara|1.9459101490553132|
|  7|       axolotl|2.0794415416798357|
|  8|      capibara|2.1972245773362196|
|  9|naked mole rat| 2.302585092994046|
| 10|      capibara|2.3978952727983707|
| 11|      capibara|2.4849066497880004|
| 12|naked mole rat|2.5649493574615367|
| 13|    flying fox| 2.639057329615259|
| 14|      capibara|  2.70805020110221|
+---+--------------+------------------+



This errors out because 

```python
math.log1p
```

is not a udf: it doesn't know how to work with strings or Column objects:

But we can transform it into a udf:

We can do the same with any function we dream up:

In [74]:
very_specific_function = functions.udf(lambda word: word[::2])

df.select('*', 
          very_specific_function('species')).show()

+---+--------------+-----------------+
| id|       species|<lambda>(species)|
+---+--------------+-----------------+
|  0|naked mole rat|          nkdml a|
|  1|      capibara|             cpbr|
|  2|       axolotl|             aool|
|  3|      capibara|             cpbr|
|  4|       axolotl|             aool|
|  5|naked mole rat|          nkdml a|
|  6|      capibara|             cpbr|
|  7|       axolotl|             aool|
|  8|      capibara|             cpbr|
|  9|naked mole rat|          nkdml a|
| 10|      capibara|             cpbr|
| 11|      capibara|             cpbr|
| 12|naked mole rat|          nkdml a|
| 13|    flying fox|            fyn o|
| 14|      capibara|             cpbr|
+---+--------------+-----------------+



If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [76]:
df.withColumn('yeah_baby',
              my_new_udf(df['id']))

DataFrame[id: smallint, species: string, yeah_baby: string]

In [79]:
typed_udf = functions.udf(math.log1p, returnType=types.FloatType())
df.withColumn('yeah_baby',
              typed_udf(df['id']))

DataFrame[id: smallint, species: string, yeah_baby: float]

In [85]:
@functions.udf('float')
def myfun(a): 
  return math.log1p(a)

df.select(myfun('id')).show()

+---------+
|myfun(id)|
+---------+
|      0.0|
|0.6931472|
|1.0986123|
|1.3862944|
| 1.609438|
|1.7917595|
|1.9459101|
|2.0794415|
|2.1972246|
|2.3025851|
|2.3978953|
|2.4849067|
|2.5649493|
|2.6390574|
|2.7080503|
+---------+



Think about this function: what is its return type?

In [ ]:
def x(a, b):
  return a + b

#### Exercise: 

Create a 'body_weight' field in our df. make it 18 for naked mole rat, 25000 for capibaras and 14 for axolotls, and 8000 for flying foxes.



In [98]:
@functions.udf('int')
def body_weight(animal):
  weights = {'naked mole rat' : 18,
             'capibara' : 25000,
             'axolotl' : 14,
             'flying fox' : 8000}

  return weights.get(animal)

df2 = df.withColumn('bw', body_weight('species'))
df2

DataFrame[id: smallint, species: string, bw: int]

If we have a column that is not the desired type, we can convert it after the fact with `.cast()`. We can also change its name using `.alias()`

In [104]:
df2.select(df2['bw'].cast(types.FloatType()).alias('yuhuuu')).show()

+-------+
| yuhuuu|
+-------+
|   18.0|
|25000.0|
|   14.0|
|25000.0|
|   14.0|
|   18.0|
|25000.0|
|   14.0|
|25000.0|
|   18.0|
|25000.0|
|25000.0|
|   18.0|
| 8000.0|
|25000.0|
+-------+



### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [106]:
df2.stat.corr('id', 'bw')

0.18822612115250825

In [108]:
df2.stat.cov('id', 'bw')

10560.857142857145

In [110]:
df2.stat.approxQuantile('bw', [.2, .4], 1e-4)

[14.0, 18.0]

### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

In [121]:
random.seed(12)

@functions.udf
def enclosure():
  return random.choice(['froopyland', 'magic kingdom'])

df3 = df2.withColumn('enclosure', enclosure())
df3.cache().show()

+---+--------------+-----+-------------+
| id|       species|   bw|    enclosure|
+---+--------------+-----+-------------+
|  0|naked mole rat|   18|magic kingdom|
|  1|      capibara|25000|magic kingdom|
|  2|       axolotl|   14|   froopyland|
|  3|      capibara|25000|   froopyland|
|  4|       axolotl|   14|magic kingdom|
|  5|naked mole rat|   18|   froopyland|
|  6|      capibara|25000|   froopyland|
|  7|       axolotl|   14|   froopyland|
|  8|      capibara|25000|magic kingdom|
|  9|naked mole rat|   18|magic kingdom|
| 10|      capibara|25000|magic kingdom|
| 11|      capibara|25000|magic kingdom|
| 12|naked mole rat|   18|magic kingdom|
| 13|    flying fox| 8000|magic kingdom|
| 14|      capibara|25000|   froopyland|
+---+--------------+-----+-------------+



In [123]:
df3.stat.crosstab('species', 'enclosure').show()

+-----------------+----------+-------------+
|species_enclosure|froopyland|magic kingdom|
+-----------------+----------+-------------+
|   naked mole rat|         1|            3|
|         capibara|         3|            4|
|       flying fox|         0|            1|
|          axolotl|         2|            1|
+-----------------+----------+-------------+



### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [131]:
df3.groupby('species')\
   .agg(functions.max('id').alias('max_id'),
        functions.min('id').alias('min_id'))\
   .show()

+--------------+------+------+
|       species|max_id|min_id|
+--------------+------+------+
|      capibara|    14|     1|
|       axolotl|     7|     2|
|    flying fox|    13|    13|
|naked mole rat|    12|     0|
+--------------+------+------+



In [128]:
df3

DataFrame[id: smallint, species: string, bw: int, enclosure: string]

GroupedData has several aggregation functions defined:

In [136]:
grouped_data = df3.groupby('species')
grouped_data.max()

DataFrame[species: string, max(id): smallint, max(bw): int]

We can do several aggregations in a single step, with a number of different syntaxes:

### Intersections

Ver much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

In [143]:
random.seed(12)

data = list(zip(random.choices(['froopyland', 'magic kingdom', 'neverland'], k = 10),
                 random.choices([3,5,7, None], k=10)))

data

other_df = spark.createDataFrame(data, schema=['enclosure', 'id'])
other_df

DataFrame[enclosure: string, id: bigint]

Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
spark.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```

In [146]:
df3.join(other_df).count() # Cross join

150

In [151]:
duplicate_columns = df3.join(other_df, on='enclosure', how='inner')
duplicate_columns.show()

+-------------+---+--------------+-----+---+
|    enclosure| id|       species|   bw| id|
+-------------+---+--------------+-----+---+
|magic kingdom| 13|    flying fox| 8000|  7|
|magic kingdom| 12|naked mole rat|   18|  7|
|magic kingdom| 11|      capibara|25000|  7|
|magic kingdom| 10|      capibara|25000|  7|
|magic kingdom|  9|naked mole rat|   18|  7|
|magic kingdom|  8|      capibara|25000|  7|
|magic kingdom|  4|       axolotl|   14|  7|
|magic kingdom|  1|      capibara|25000|  7|
|magic kingdom|  0|naked mole rat|   18|  7|
|magic kingdom| 13|    flying fox| 8000|  7|
|magic kingdom| 12|naked mole rat|   18|  7|
|magic kingdom| 11|      capibara|25000|  7|
|magic kingdom| 10|      capibara|25000|  7|
|magic kingdom|  9|naked mole rat|   18|  7|
|magic kingdom|  8|      capibara|25000|  7|
|magic kingdom|  4|       axolotl|   14|  7|
|magic kingdom|  1|      capibara|25000|  7|
|magic kingdom|  0|naked mole rat|   18|  7|
|magic kingdom| 13|    flying fox| 8000|  3|
|magic kin

In [155]:
duplicate_columns.select(other_df['id']).show()

+---+
| id|
+---+
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  7|
|  3|
|  3|
+---+
only showing top 20 rows



In [150]:
df3.join(other_df, on=['enclosure', 'id'], how='inner').show()

+----------+---+--------------+-----+
| enclosure| id|       species|   bw|
+----------+---+--------------+-----+
|froopyland|  5|naked mole rat|   18|
|froopyland|  3|      capibara|25000|
|froopyland|  3|      capibara|25000|
+----------+---+--------------+-----+



#### Digression

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

In Google Colab, we need to do additional work to expose it. We need a Spark session with a web UI port other than 4040. In this example we are using 4050.


```python
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
```        
        

We can erase it with `unpersist`

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each creature's body weight for their enclosure.

In [157]:
df3.show()

+---+--------------+-----+-------------+
| id|       species|   bw|    enclosure|
+---+--------------+-----+-------------+
|  0|naked mole rat|   18|magic kingdom|
|  1|      capibara|25000|magic kingdom|
|  2|       axolotl|   14|   froopyland|
|  3|      capibara|25000|   froopyland|
|  4|       axolotl|   14|magic kingdom|
|  5|naked mole rat|   18|   froopyland|
|  6|      capibara|25000|   froopyland|
|  7|       axolotl|   14|   froopyland|
|  8|      capibara|25000|magic kingdom|
|  9|naked mole rat|   18|magic kingdom|
| 10|      capibara|25000|magic kingdom|
| 11|      capibara|25000|magic kingdom|
| 12|naked mole rat|   18|magic kingdom|
| 13|    flying fox| 8000|magic kingdom|
| 14|      capibara|25000|   froopyland|
+---+--------------+-----+-------------+



1) Calculate the mean and std of hitpoints for each location

2) Annotate each creature with the stats corresponding to their location

3) Calculate the z-score

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

Once registered, we can perform queries as complex as we want.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

## Writing out


#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

2) Total ticket amounts per origin

3) Top 10 Airlines by average amount



## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf